In [1]:
import sys # aby importy z dołu działały
sys.path.append(r"D:\MyProjects_4Fun\projects\World of Warcraft\python-etl")

In [2]:
from sqlalchemy import text
from moduly.db_core import utworz_engine_do_db, _czy_duplikat
from moduly.services_persist_wynik import przefiltruj_dane_misji, zapisz_misje_dialogi_ai_do_db
from moduly.ai_gemini import zaladuj_api_i_klienta, instrukcja_tlumacz, misje_dialogi_po_polsku_zapisz_do_db, instrukcja_redaktor
from sqlalchemy.exc import IntegrityError
from scraper_wiki_main import parsuj_misje_z_url
import json
import zlib
import base64
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint

In [3]:
silnik = utworz_engine_do_db()

In [4]:
klient = zaladuj_api_i_klienta("API_TLUMACZENIE")

q_select_tresc = text("""
WITH hashe AS (
    SELECT 
        m.MISJA_ID_MOJE_PK,
        z.HTML_SKOMPRESOWANY,
        ROW_NUMBER() OVER (
            PARTITION BY z.MISJA_ID_MOJE_FK 
            ORDER BY z.DATA_WYSCRAPOWANIA DESC
        ) AS r
    FROM dbo.ZRODLO AS z
    INNER JOIN dbo.MISJE AS m
    ON z.MISJA_ID_MOJE_FK = m.MISJA_ID_MOJE_PK
    WHERE 1=1
    AND m.MISJA_ID_Z_GRY IS NOT NULL
    AND m.MISJA_ID_Z_GRY <> 123456789
                        
    AND m.MISJA_ID_MOJE_PK = :id_misji
                        
    AND NOT EXISTS (
        SELECT 1
        FROM dbo.MISJE_STATUSY AS ms
        WHERE ms.MISJA_ID_MOJE_FK = m.MISJA_ID_MOJE_PK
        AND ms.STATUS = N'1_PRZETŁUMACZONO'
    )
)
SELECT 
    MISJA_ID_MOJE_PK, HTML_SKOMPRESOWANY
FROM hashe
WHERE r = 1
ORDER BY MISJA_ID_MOJE_PK
;
""")

q_select_npc = text("""
WITH wszystkie_idki AS (
    SELECT
        tabela_wartosci.ID_NPC
    FROM dbo.MISJE AS m
    CROSS APPLY (
        VALUES
            (m.NPC_START_ID),
            (m.NPC_KONIEC_ID)
    ) AS tabela_wartosci (ID_NPC)
    WHERE m.MISJA_ID_MOJE_PK = :misja_id

    UNION

    SELECT
        ds.NPC_ID_FK
    FROM dbo.DIALOGI_STATUSY AS ds
    WHERE ds.MISJA_ID_MOJE_FK = :misja_id
),

oczyszczone_dane AS (
    SELECT
        wi.ID_NPC,
        ns.STATUS,
        CASE
            WHEN CHARINDEX('[', ns.NAZWA) > 0
            THEN RTRIM(LEFT(ns.NAZWA, CHARINDEX('[', ns.NAZWA) - 1))
            ELSE ns.NAZWA
        END AS CZYSTA_NAZWA
    FROM wszystkie_idki AS wi
    INNER JOIN dbo.NPC_STATUSY AS ns
        ON wi.ID_NPC = ns.NPC_ID_FK
)

SELECT DISTINCT
    pvt.[0_ORYGINAŁ],
    pvt.[3_ZATWIERDZONO]
FROM oczyszczone_dane
PIVOT (
    MAX(CZYSTA_NAZWA)
    FOR STATUS IN ([0_ORYGINAŁ], [3_ZATWIERDZONO])
) AS pvt
;
""")

q_select_slowa_kluczowe = text("""
    SELECT 
        sk.SLOWO_EN,
        sk.SLOWO_PL
    FROM dbo.MISJE_SLOWA_KLUCZOWE AS msk
    INNER JOIN dbo.SLOWA_KLUCZOWE AS sk
    ON msk.SLOWO_ID = sk.SLOWO_ID_PK
    WHERE msk.MISJA_ID_MOJE_FK = :misja_id
""")

parametry = {"id_misji": 6}

with silnik.connect() as conn:
    wyniki_z_bazy = conn.execute(q_select_tresc, parametry).mappings().all()

    print(f"Znaleziono rekordów: {len(wyniki_z_bazy)}\n")
    
    for wiersz in wyniki_z_bazy:
        misja_id = wiersz["MISJA_ID_MOJE_PK"]
        zakodowane_dane = wiersz["HTML_SKOMPRESOWANY"]

        npc_z_bazy = conn.execute(q_select_npc, {"misja_id": misja_id}).all()
        slowa_kluczowe_z_bazy = conn.execute(q_select_slowa_kluczowe, {"misja_id": misja_id}).all()

        if not zakodowane_dane:
            print(f"SKIP [ID: {misja_id}] - Brak skompresowanego HTML w bazie.")
            continue

        try:
            skompresowane_bajty = base64.b64decode(zakodowane_dane)
            tekst_html = zlib.decompress(skompresowane_bajty).decode("utf-8")

            surowe_dane = parsuj_misje_z_url(None, html_content=tekst_html)
            przetworzone_dane = przefiltruj_dane_misji(surowe_dane, jezyk="EN")

            wsad_dla_geminisia_npc = set(npc for npc in npc_z_bazy)
            wsad_dla_geminisia_sk = set(slowo for slowo in slowa_kluczowe_z_bazy)

            wsad_dla_geminisia_cialo = json.dumps(przetworzone_dane, indent=4, ensure_ascii=False)

            npc_tekst = "\n".join([f"- {n[0]} -> {n[1]}" for n in wsad_dla_geminisia_npc if n[0] and n[1]])
            sk_tekst = "\n".join([f"- {k[0]} -> {k[1]}" for k in wsad_dla_geminisia_sk if k[0] and k[1]])

            try:
                # --- ETAP 1: TŁUMACZENIE ---
                print(f"--- [ID: {misja_id}] Etap 1: Tłumaczenie... ---")
                odpowiedz_01 = klient.models.generate_content(
                    model="gemini-3-pro-preview",
                    contents=wsad_dla_geminisia_cialo,
                    config={
                        "system_instruction": instrukcja_tlumacz(npc_tekst, sk_tekst),
                        "response_mime_type": "application/json"
                    }
                )

                przetlumaczone = json.loads(odpowiedz_01.text)
                
                zapisz_misje_dialogi_ai_do_db(
                    silnik=silnik, 
                    misja_id=misja_id, 
                    przetlumaczone=przetlumaczone, 
                    status="1_PRZETŁUMACZONO"
                )

                # --- ETAP 2: REDAKCJA ---
                print(f"--- [ID: {misja_id}] Etap 2: Redakcja... ---")
                
                wsad_do_redakcji = json.dumps(przetlumaczone, indent=4, ensure_ascii=False)

                odpowiedz_02 = klient.models.generate_content(
                    model="gemini-3-pro-preview",
                    contents=wsad_do_redakcji,
                    config={
                        "system_instruction": instrukcja_redaktor(wsad_dla_geminisia_cialo, npc_tekst, sk_tekst),
                        "response_mime_type": "application/json"
                    }
                )

                zredagowane = json.loads(odpowiedz_02.text)

                zapisz_misje_dialogi_ai_do_db(
                    silnik=silnik, 
                    misja_id=misja_id, 
                    przetlumaczone=zredagowane,
                    status="2_ZREDAGOWANO"
                )

            except Exception as er:
                print(f"BŁĄD w tłumaczeniu/zapisie misji: {misja_id} --> {er}")

        except Exception as e:
            print(f"BŁĄD ogólny przy ID {misja_id}: {e}")

KLUCZ ZWARTY I GOTOWY!
Znaleziono rekordów: 1

--- [ID: 6] Etap 1: Tłumaczenie... ---

--- [START] Zapis misji ID: 6 | Status: 1_PRZETŁUMACZONO ---
-> Przygotowano danych misji: 14 wierszy.
-> Zaktualizowano tytuł na: 'Pułapka Łowcy'
-> Rozpoczynam mapowanie NPC w dialogach...
-> Przygotowano dialogów: 3 wierszy.
-> COMMIT: Dane zostały wysłane do bazy.
--- [KONIEC] Sukces dla misji ID: 6 ---

--- [ID: 6] Etap 2: Redakcja... ---

--- [START] Zapis misji ID: 6 | Status: 2_ZREDAGOWANO ---
-> Przygotowano danych misji: 14 wierszy.
-> Zaktualizowano tytuł na: 'Pułapka Łowcy'
-> Rozpoczynam mapowanie NPC w dialogach...
-> Przygotowano dialogów: 3 wierszy.
-> COMMIT: Dane zostały wysłane do bazy.
--- [KONIEC] Sukces dla misji ID: 6 ---



In [5]:
przetlumaczone

{'Misje_PL': {'Podsumowanie_PL': {'Tytuł': 'Pułapka Łowcy'},
  'Cele_PL': {'Główny': {'1': 'Znajdź w okolicy części potrzebne do stworzenia Pułapki Zamrażającej.'},
   'Podrzędny': {'1': 'Drobina lodu odebrana od Herberta Gloombursta',
    '2': '[Zardzewiały Łańcuch]',
    '3': '[Stare Źródło]'}},
  'Treść_PL': {'1': 'Jeśli chcesz nazywać się prawdziwym łowcą, musisz opanować sztukę zastawiania pułapek.',
   '2': 'Na szczęście tak się składa, że jestem w tym ekspertem.',
   '3': 'Najpierw musisz zebrać zapasy z okolicy, aby sporządzić porządną pułapkę zamrażającą.',
   '4': 'Herbert może zaopatrzyć cię w drobinę lodu. Przeszukaj okolicę w poszukiwaniu pozostałych materiałów.',
   '5': 'Będziesz potrzebować tej pułapki, aby poradzić sobie z innym problemem, który niedawno się pojawił.'},
  'Postęp_PL': {'1': 'Zdobyłeś potrzebne zapasy? Herbert powinien mieć drobinę lodu, a pozostałe dwa elementy muszą być gdzieś w pobliżu.'},
  'Zakończenie_PL': {'1': 'Przy odrobinie pracy powstanie z t

In [6]:
zredagowane

{'Misje_PL': {'Podsumowanie_PL': {'Tytuł': 'Pułapka Łowcy'},
  'Cele_PL': {'Główny': {'1': 'Przeszukaj okolicę, aby znaleźć części potrzebne do stworzenia Pułapki Zamrażającej.'},
   'Podrzędny': {'1': 'Drobina lodu odebrana od Herberta Gloombursta',
    '2': '[Zardzewiały Łańcuch]',
    '3': '[Stare Źródło]'}},
  'Treść_PL': {'1': 'Jeśli chcesz uchodzić za prawdziwego łowcę, musisz opanować sztukę zastawiania sideł.',
   '2': 'Na szczęście trafiłeś na eksperta w tej dziedzinie.',
   '3': 'Najpierw zbierz z okolicy materiały, z których sporządzisz solidną Pułapkę Zamrażającą.',
   '4': 'Herbert powinien mieć drobinę lodu. Rozejrzyj się w pobliżu za pozostałymi częściami.',
   '5': 'Pułapka będzie ci niezbędna, by poradzić sobie z pewnym problemem, który niedawno się pojawił.'},
  'Postęp_PL': {'1': 'Masz już niezbędne zapasy? Herbert powinien mieć drobinę lodu, a pozostałe dwa elementy muszą leżeć gdzieś w pobliżu.'},
  'Zakończenie_PL': {'1': 'Przy odrobinie wysiłku powstanie z tego s